In [1]:
import os
import glob
import random
from sklearn.model_selection import train_test_split

In [2]:
import librosa
import numpy as np

In [6]:
!pip install audiomentations

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl.metadata (60 kB)
Using cached numpy-2.2.6-cp310-cp310-win_amd64.whl (12.9 MB)
Failed to build numpy-minmax


  error: subprocess-exited-with-error
  
  exit code: 1
  
  [65 lines of output]
  C:\Users\letta\AppData\Local\Temp\pip-build-env-952xwez9\overlay\Lib\site-packages\setuptools_scm\_integration\version_inference.py:51: UserWarning: version of None already set
    warnings.warn(self.message)
  C:\Users\letta\AppData\Local\Temp\pip-build-env-952xwez9\overlay\Lib\site-packages\setuptools\config\_apply_pyprojecttoml.py:82: SetuptoolsDeprecationWarning: `project.license` as a TOML table is deprecated
  !!
  
          ********************************************************************************
          Please use a simple string containing a SPDX expression for `project.license`. You can also use `project.license-files`. (Both options available on setuptools>=77.0.0).
  
          By 2027-Feb-18, you need to update your project and remove deprecated calls
          or your builds will no longer be supported.
  
          See https://packaging.python.org/en/latest/guides/writing-pyproj

In [8]:
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift

ModuleNotFoundError: No module named 'audiomentations'

In [ ]:
import torch from torch.utils.data import Dataset

In [3]:
def split_dataset(root_dir, test_size=0.15, val_size=0.15):
    files = glob.glob(os.path.join(root_dir, "**/*.wav"), recursive=True)
    labels = [1 if "barbie" in f else 0 for f in files]

    X_train, X_temp, y_train, y_temp = train_test_split(
        files, labels, test_size=test_size + val_size, stratify=labels, random_state=42
    )

    rel_val = val_size / (test_size + val_size)

    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=rel_val, stratify=y_temp, random_state=42
    )

    return (X_train, y_train), (X_val, y_val), (X_test, y_test)


In [4]:
def extract_logmel(path, sr=16000, n_mels=64):
    y, _ = librosa.load(path, sr=sr)
    mel = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=n_mels)
    logmel = librosa.power_to_db(mel)
    return logmel


In [ ]:
def extract_cqt(path, sr=16000, bins=84):
    y, _ = librosa.load(path, sr=sr)
    cqt = librosa.cqt(y, sr=sr, n_bins=bins)
    cqt_db = librosa.amplitude_to_db(np.abs(cqt))
    return cqt_db


In [ ]:
def extract_mfcc_delta(path, sr=16000, n_mfcc=20):
    y, _ = librosa.load(path, sr=sr)
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    delta = librosa.feature.delta(mfcc)
    delta2 = librosa.feature.delta(mfcc, order=2)
    return np.vstack([mfcc, delta, delta2])


In [ ]:
augment = Compose([
    AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.5),
    TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
    PitchShift(min_semitones=-2, max_semitones=2, p=0.5),
    Shift(p=0.5),
])

def load_and_augment(path, sr=16000):
    y, _ = librosa.load(path, sr=sr)
    y_aug = augment(samples=y, sample_rate=sr)
    return y_aug
